In [ ]:
##Model for One-million dataset
x = tf.placeholder('int32', [batch_size, max_sent_size], name="x")
y = tf.placeholder('int32', [batch_size], name="y")
x_mask  = tf.placeholder('bool', [batch_size, max_sent_size], name='x_mask') 
is_train = tf.placeholder('bool', [], name='is_train')
word_emb_mat = tf.placeholder('float', [None, word_emb_size], name='emb_mat')
input_keep_prob = tf.cond(is_train,lambda:keep_prob, lambda:tf.constant(1.0))
x_len = tf.reduce_sum(tf.cast(x_mask, 'int32'), 1)

with tf.name_scope("word_embedding"):
    if mode == 'train':
        unk_word_emb_mat = tf.get_variable("word_emb_mat", dtype='float', shape=[unk_vocab_size, word_emb_size], initializer=tf.contrib.layers.xavier_initializer(uniform=False, seed=0, dtype=tf.float32))
    else:
        unk_word_emb_mat = tf.get_variable("word_emb_mat", shape=[unk_vocab_size, word_emb_size], dtype='float')
        
    final_word_emb_mat = tf.concat([word_emb_mat, unk_word_emb_mat], 0)
    Wx = tf.nn.embedding_lookup(final_word_emb_mat, x) 
    
    
### First Bidirectional Lstm starts
with tf.variable_scope("lstm1"):
    cell_fw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
    cell_bw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

    d_cell_fw1 = tf.contrib.rnn.DropoutWrapper(cell_fw1, input_keep_prob=input_keep_prob)
    d_cell_bw1 = tf.contrib.rnn.DropoutWrapper(cell_bw1, input_keep_prob=input_keep_prob)
    
    (fw_h1, bw_h1), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw1, d_cell_bw1, Wx, sequence_length=x_len, dtype='float', scope='lstm1')
    h = tf.concat([fw_h1, bw_h1], 2)

## First bidirectional Lstm ends
    


## Second Bidirectional lstm starts

with tf.variable_scope("lstm3"):
    cell_fw3 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
    cell_bw3 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

    d_cell_fw3 = tf.contrib.rnn.DropoutWrapper(cell_fw3, input_keep_prob=input_keep_prob)
    d_cell_bw3 = tf.contrib.rnn.DropoutWrapper(cell_bw3, input_keep_prob=input_keep_prob)
    
    (fw_h3, bw_h3), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw3, d_cell_bw3, h, sequence_length=x_len, dtype='float', scope='lstm3')
    h3 = tf.concat([fw_h3, bw_h3], 2)
    
## Second Bidirectional lstm ends


def attention(input_x, input_mask, W_att):
    h_masked = tf.boolean_mask(input_x, input_mask)
    h_tanh = tf.tanh(h_masked)
    u = tf.matmul(h_tanh, W_att)
    a = tf.nn.softmax(u)
    c = tf.reduce_sum(tf.multiply(h_tanh, a), 0)  
    return c

with tf.variable_scope("attention"):
    W_att = tf.Variable(tf.truncated_normal([2*hidden_size, 1], mean=0.0, stddev=0.1, seed=0), name="W_att")
    c = tf.expand_dims(attention(h3[0], x_mask[0], W_att), 0)
    for i in range(1, batch_size):
        c = tf.concat([c, tf.expand_dims(attention(h3[i], x_mask[i], W_att), 0)], 0)
        
with tf.variable_scope("softmax_layer"):
    W = tf.Variable(tf.truncated_normal([2*hidden_size, num_POS], mean=0.0, stddev=0.1, seed=0), name="W")
    b = tf.Variable(tf.zeros([num_POS]), name="b")
    drop_h = tf.nn.dropout(h, input_keep_prob)
    logits_h = tf.matmul(drop_h, W) + b
    pos_tag = tf.argmax(logits_h, 1)
    
    W1 = tf.Variable(tf.truncated_normal([2*hidden_size, num_senses], mean=0.0, stddev=0.1, seed=0), name="W1")
    b1 = tf.Variable(tf.zeros([num_senses]), name="b1")
    drop_c = tf.nn.dropout(c, input_keep_prob)
    logits = tf.matmul(drop_c, W1) + b1
    sense = tf.argmax(logits, 1)
    
    predictions = tf.concat([pos_tag, sense],2)
    